# Ising模型PEC和ZNE实验

2D横场Ising模型作为测试的实验线路之一。该模型除了在物理中有着充分应用场景外，对应线路还具有线路深度浅、适配芯片拓扑结构，以及难以进行经典近似等特点，适合作为量子计算的实用性测试。

我们考察如下形式的Ising模型：
$$
H = \sum_{(i,j)\in E} JZ_i Z_j + \sum_i h_i X_i
$$
为模拟该哈密顿量，我们使用Trotter分解的方式，将其分解为$(\exp({-iJH_{ZZ}t/s})\exp(-ihH_Xt/s))^s$的形式。为方便起见，我们固定$t/s=dt$，则增加Trotter层数即增加实际演化时长$t = sdt$，并通过一些方法寻找其动力学特征。

这里我们选择的是Z方向平均期望$\langle Z \rangle = \frac{1}{N}\sum_{i}\langle Z_i\rangle$和Y方向平均期望（公式同上）两个值。其中Y方向期望可以直接在线路末端对应比特添加RX(-pi/2)门获得。将两个点在平面直角坐标系上画出来，随着Trotter层数增加，我们可以画出一个环。通过对比无噪音模拟结果和错误缓解结果就可以比较芯片和算法性质。

在实际执行量子算法时，求期望一般是最常见的功能，许多NISQ算法，如QAOA、动力学模拟、VQE等都涉及到这一项。然而，受到噪音影响，一般直接运行量子线路无法获得准确的期望值。因此，学术界和工业界开发出了一些方案，通过间接方式，以更高的线路运行次数为代价，获得尽可能接近原始结果的期望。这种操作就被称作错误缓解。我们这里引入的是两种常见方案，分别叫做零噪音外推和概率错误消除。

零噪音外推实验的思路为，通过某种方式操控噪音大小。假设什么都不做的噪音为1，通过某些方式放大噪音，可以想象期望值会按照一定规律减小，直到达到大约0（即完全混态）的程度。那么，通过这样的方式，我们就可以反向外推噪音为0的情况。参考文献[1]，Sparse Pauli-Lindberg模型假设噪音表现形式和以一定概率随机在线路上作用泡利门的平均是一致的，因此在通过噪音学习获得对应噪音系数之后，我们可以推算出若想噪音放大到某些倍数，则可以以什么样的概率作用对应泡利门在量子比特上，进而获得近似的噪音放大结果。

概率错误消除的思路和零噪音外推相似，只不过不再通过放大噪音并反向外推的方式，而是直接假设存在一个逆噪音信道，和原噪音抵消。或者说，我们将噪音放大倍数设置为-1。然而，当倍数设置为-1时，获得的“概率”将存在[0,1]区间之外的数。对此的解决方案是，我们提取出每个“概率”的符号，以及所有“概率”绝对值的和。这样，剩下的内容就是真正的概率了，求出来的结果不是直接求平均，而是要乘以对应符号之后再平均，并乘上之前提取出来的和。但通过这样的提取方案，会增加概率的方差，因此我们需要进行更多的采样，才能使结果保持稳定。具体推导可见文献[2]

除了线路误差外，读取误差也会有很大影响。这里我们引入的是被称作模型无关的噪音缓解模型，参见文献[3]。该方法的核心思想认为，对于任意Z方向的投影测量期望，读取误差对其影响仅为在原始结果上乘以一个系数。该系数和具体线路无关，只和对应量子比特以及投影测量的字符串有关。因此，我们可以先生成一组空线路和对应投影测量的值，并在实际测量的时候调用、计算对应系数。

首先引入需要的模块，包括文件处理等基本处理模块，Ising模型模块，ZNE（PER，即概率错误增加）和PEC对应的模块，函数拟合模块等。

In [ ]:
from originbench.error_mitigation.applications.ising import *
from originbench.error_mitigation.algorithm.pec_experiment import PEC
from originbench.error_mitigation.algorithm.per_experiment import PER
from originbench.error_mitigation.algorithm.fit import *

In [ ]:
import json
import matplotlib.pyplot as plt
%matplotlib inline

cloud_api_key = (
    'your api token'
)

构建需要的Ising模型。这里应该和噪音学习的CZ pattern和连接性一样。注意，在构建模型时，应先以逻辑比特进行构建，然后输入逻辑比特：物理比特的映射关系。

In [ ]:
pattern1 = [[0,1], [2,3], [4,5], [6,7]]
pattern2 = [[1,2], [3,4], [5,6], [7,8]]
cz_pattern = [pattern1, pattern2]
mapping = {0:12,1:6,2:0, 3:1, 4:2, 5:3, 6:4, 7:10, 8:16}
coupling_map = sum(cz_pattern, [])
model = ising_model(topo=coupling_map, cz_pattern=cz_pattern, special=False)

导入噪音学习的结果，并将专门的CZ pattern和coefficient存在noise_model中。噪音学习可见另一文件。

In [ ]:
with open(r'output_noise_data/noise_data.json', 'r') as f:
    noise_data = json.load(f)
noise_model = []
for key, value in noise_data.items():
    noise_model.append([json.loads(key), value[0]['coeffs']])

获取量子计算后端

In [ ]:
from originbench.error_mitigation.backends.backends import qcloud_run, run

运行模型无关噪音缓解方案，输入对应量子比特、采用次数，并运行线路。

In [ ]:
readout_model_real = ReadoutMitigation(mapping.values())
readout_model_real.generate_circuits(sample=1)
readout_model_real.run(qcloud_run, auto_mapping=False, measurement_amend=False, key=cloud_api_key)

In [ ]:
for i in range(2**9):
    j = [int(k) for k in np.binary_repr(i, 9)]
    print(j, readout_model_real.get_d0(j))

在ZNE中，要首先确定对应噪音放大倍数

In [ ]:
strength_list = [1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6]

以一个线路为例，先生成一个线路，例如Y方向，Trotter层数为1的情况。
ZNE的基本思路为：
1. 构建PER实例对象，要输入对应的逻辑线路、CZ pattern和映射信息；
2. 生成对应实验（即线路和后处理信息），包括噪音参数、强度放大倍数、采样次数；
3. 运行线路，输入运行后端和各种运行参数；
4. 后处理结果（包括求期望、读取错误缓解等），输入哈密顿量Dict（目前仅接受Z方向，以{pauli:coeff}的方式）和读取错误缓解模型。
5. 进行零噪音拟合。输入拟合函数，输出为零噪音外推结果、拟合参数不确定性、各噪音强度下的期望，以及拟合参数本身。

In [ ]:
ir_y = model.generate_measure_base(h=1, J=0.15, dt=1/4, base='Y', trotter_layer=3)
per_instance_y = PER(ir=ir_y, cz_pattern=cz_pattern, map=mapping)
per_instance_y.generate_experiment(noise_model=noise_model, strength_list=strength_list, sample=1)
per_instance_y.run_experiment(qcloud_run, measurement_amend=False, key=cloud_api_key, shots=200)
per_instance_y.convert_result(mz_exp(9), readout_model_real)
zero_y, unctty, ydata, popt = per_instance_y.zne(new_exp)

同样的，我们可以看一下无噪音的模拟结果。run函数是CPUQVM。

In [ ]:
def calculate_theory(ir, expectation_dict):
    result = run([ir])[0]
    measurement_error=None
    f = 0
    for exp, coeff in expectation_dict.items():
        e = 0
        if measurement_error is None:
            m = 1
        else:
            if len(measurement_error.result) == 0:
                pass
            exp = np.array([int(i) for i in exp])
            m = measurement_error.get_d0(exp)
        for key, value in result.items():
            exp = np.array([int(i) for i in exp][::-1])
            key = np.array([int(i) for i in key][::-1])
            # key = np.array([int(i) for i in key])
            e += (1 - 2 * (np.sum(key * exp) % 2)) * value
        f += e / m * coeff / sum(result.values())
    return f
theory_y = calculate_theory(ir_y, mz_exp(9))

画出实际各个点的测量值（和外推值）、拟合函数曲线，并特别标注出无噪音模拟的点。

In [ ]:
%matplotlib inline
plt.scatter([0]+strength_list, [zero_y]+list(ydata))
plt.plot([0]+strength_list, new_exp(np.array([0]+strength_list), *popt))
plt.plot([0], theory_y, '*')
plt.show()

接下来是完整流程，Trotter层数从1到14层，Z和Y两个方向。计算ZNE结果和模拟结果并储存。

In [ ]:
z_zne = []
y_zne = []
z_theory = []
y_theory = []

for trotter in range(1, 2):
    ir_y = model.generate_measure_base(h=1, J=0.15, dt=1/4, base='Y', trotter_layer=trotter)
    ir_z = model.generate_measure_base(h=1, J=0.15, dt=1/4, base='Z', trotter_layer=trotter)
    
    per_instance_y = PER(ir=ir_y, cz_pattern=cz_pattern, map=mapping)
    per_instance_y.generate_experiment(noise_model=noise_model, strength_list=strength_list, sample=5)
    per_instance_y.run_experiment(qcloud_run, measurement_amend=False, key=cloud_api_key, shots=200)
    per_instance_y.convert_result(mz_exp(9), readout_model_real)
    zero_y, _, _, _ = per_instance_y.zne(func=new_exp,)

    per_instance_z = PER(ir=ir_z, cz_pattern=cz_pattern, map=mapping)
    per_instance_z.generate_experiment(noise_model=noise_model, strength_list=strength_list, sample=5)
    per_instance_z.run_experiment(qcloud_run, measurement_amend=False, key=cloud_api_key, shots=200)
    per_instance_z.convert_result(mz_exp(9), readout_model_real)
    zero_z, _, _, _ = per_instance_z.zne(func=new_exp,)

    z_zne.append(-zero_z)
    y_zne.append(-zero_y)

    z = calculate_theory(ir_z, mz_exp(9))
    y = calculate_theory(ir_y, mz_exp(9))
    z_theory.append(z)
    y_theory.append(y)

    print(-zero_z, z, -zero_y, y)

    print('trotter', trotter)

然后画图

In [ ]:
%matplotlib inline
plt.figure(figsize=(8, 6))
plt.plot(z_theory, y_theory, 'o', label='simulation')
plt.plot(z_zne, y_zne, 'x', label='zne')
plt.legend()
plt.show()

PEC同理，通过生成示例、构造实验、运行线路、处理结果。但sample数量最好大一些。

In [ ]:
z_pec = []
y_pec = []

for trotter in range(1, 2):
    ir_y = model.generate_measure_base(h=1, J=0.15, dt=1/4, base='Y', trotter_layer=trotter)
    ir_z = model.generate_measure_base(h=1, J=0.15, dt=1/4, base='Z', trotter_layer=trotter)
    
    pec_instance_y = PEC(ir=ir_y, cz_pattern=cz_pattern, map=mapping)
    pec_instance_y.generate_experiment(noise_model=noise_model, sample=1 * trotter)
    pec_instance_y.run_experiment(qcloud_run, measurement_amend=False, key=cloud_api_key, shots=200)
    zero_y = pec_instance_y.convert_result(mz_exp(9), readout_model_real)

    pec_instance_z = PEC(ir=ir_z, cz_pattern=cz_pattern, map=mapping)
    pec_instance_z.generate_experiment(noise_model=noise_model, sample=1 * trotter)
    pec_instance_z.run_experiment(qcloud_run, measurement_amend=False, key=cloud_api_key, shots=200)
    zero_z = pec_instance_z.convert_result(mz_exp(9), readout_model_real)

    z_pec.append(-zero_z)
    y_pec.append(-zero_y)

In [ ]:
%matplotlib inline
plt.figure(figsize=(8, 6))
plt.plot(z_theory, y_theory, 'o', label='simulation')
plt.plot(z_pec, y_pec, 'x', label='zne')
plt.legend()
plt.show()

# 参考文献
[1] Berg, E. van den, Minev, Z. K., Kandala, A. & Temme, K. Probabilistic error cancellation with sparse Pauli-Lindblad models on noisy quantum processors. Preprint at http://arxiv.org/abs/2201.09866 (2022). 

[2] Endo, S., Benjamin, S. C. & Li, Y. Practical Quantum Error Mitigation for Near-Future Applications. Phys. Rev. X 8, 031027 (2018). 

[3] Berg, E. van den, Minev, Z. K. & Temme, K. Model-free readout-error mitigation for quantum expectation values. Phys. Rev. A 105, 032620 (2022). 